In [1]:
import numpy as np 
import cv2 
import tensorflow as tf
import keras
import os 
from random import shuffle 
import matplotlib.pyplot as plt
from keras.models import load_model
from tqdm import tqdm
import pickle
import tensorflow as tf
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 
import win32com.client as wincl
speak = wincl.Dispatch("SAPI.SpVoice")
from imutils.perspective import four_point_transform

Using TensorFlow backend.


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
MODEL_NAME = 'model3.tflearn'
IMG_SIZE = 80
#LR = 1e-3
LR = 0.00146

In [3]:
train_data = np.load('train_data3.npy') 
test_data = np.load('test_data3.npy') 

In [4]:
tf.reset_default_graph() 
convnet = input_data(shape =[None, IMG_SIZE, IMG_SIZE,3], name ='input')   
#conv_2d(convnet, weight, SIZE OF MASK, activation ='relu')

convnet = conv_2d(convnet, 32, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 3) 
#convnet = dropout(convnet, 0.3)

convnet = conv_2d(convnet, 64, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 3)
#convnet = dropout(convnet, 0.3)
  
convnet = conv_2d(convnet, 128, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 3)
#convnet = dropout(convnet, 0.3)
  
convnet = conv_2d(convnet, 64, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 3)
#convnet = dropout(convnet, 0.3)
  

convnet = fully_connected(convnet, 1024, activation ='relu') 
convnet = dropout(convnet, 0.3)
  
convnet = fully_connected(convnet, 5, activation ='softmax')

convnet = regression(convnet, optimizer ='adam', learning_rate = LR, 
      loss ='categorical_crossentropy', name ='targets') 
  
model = tflearn.DNN(convnet) 
  

model.load(MODEL_NAME)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\jason12\model3.tflearn


In [5]:
IMG_SIZE1 = 1000
old = 'nothing'
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('C:/Users/jason12/Desktop/work/dataset/predict/video/1.mp4')
while(1):
    _,im = cap.read()
    im = cv2.resize(im, (IMG_SIZE1, IMG_SIZE1))

    hsv_img = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    COLOR_MIN = np.array([169, 100, 100],np.uint8)
    COLOR_MAX = np.array([189, 255, 255],np.uint8)
    frame_threshed = cv2.inRange(hsv_img, COLOR_MIN, COLOR_MAX)

    imgray = frame_threshed
    ret,thresh = cv2.threshold(frame_threshed,127,255,0)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    # Find the index of the largest contour
    areas = [cv2.contourArea(c) for c in contours]
    if areas == []:
        continue
    else:    
        max_index = np.argmax(areas)
        cnt=contours[max_index]
        #make the bounding rectangle
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(im,(x,y),(x+w,y+h),(0,255,0),2)
        
        #crop the rectangle
        crop_img = im[y:y+h, x:x+w] #this is a rgb image
        cv2.imwrite('sign.jpg', crop_img) #this is a rgb image

        test = cv2.resize(crop_img,(80,80))
        model_out = model.predict([test])
        #print(model_out)
        index = np.argmax(model_out)

        if index == 0: 
            ans = 'no honking sign ahead'
            print(ans)
            if old != ans:
                speak.Speak('no honking sign ahead')
            old = ans
            
        elif index == 1:
            ans = 'School sign ahead'
            print(ans)
            if old != ans:
                speak.Speak('School sign ahead')
            old = ans
            
        elif index == 2:
            ans = 'stop sign ahead'
            print(ans)
            if old != ans:
                speak.Speak('stop sign ahead')
            old = ans
            
        elif index == 3:
            ans = 'no_u_turn'
            print(ans)
            if old != ans:
                speak.Speak('no you turn')
            old = ans
            
        elif index == 4:
            ans = 'no_left'
            print(ans)
            if old != ans:
                speak.Speak('no left')
            old = ans
            
        #else:
         #   ans = 'Cannot classify'
        #print(ans)
        cv2.imshow("Show",im)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()   

no honking sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
no_left
no_left
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School sign ahead
School s